In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
import h5py
import time
import pickle
import warnings

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

from Source.util import getPathsToVisit
from Source.ipythonWidgets import overlayViewer, reconViewer
from Source.featureExtractor import extract_features, extract_features_and_labels

In [3]:
pathsToVisit = getPathsToVisit('../ML/Train')
print('Number of subjects:', len(pathsToVisit))

features_train = []
labels_train = []
for subjectPath in pathsToVisit:
    print(subjectPath)

    # Load the dataset.
    reconPath = os.path.join(subjectPath, 'recon4d.hdf5')
    with h5py.File(reconPath) as f:
        spacing = np.array(f['spacing'])
        temp_res = np.array(f['temp_res']).astype(np.float)/1000
        recon = np.array(f['recon'])
    ss_f = [2, 2, 2, 1]  # Subsampling factor [x,y,z,t]

    recon = recon[::ss_f[0],::ss_f[1],::ss_f[2],::ss_f[3]]
    spacing = np.array([ss_f[0],ss_f[1],ss_f[2]])*spacing
    nx, ny, nz, nt = recon.shape
    
    # Load the labels.
    labelsPath = os.path.join(subjectPath, 'labels.hdf5')
    with h5py.File(labelsPath) as f:
        labels3d = np.array(f['labels'])
    labels3d = labels3d[::ss_f[0],::ss_f[1],::ss_f[2]]
    
    time_stamps = np.arange(nt)*temp_res
    features, labels = extract_features_and_labels(recon, spacing, time_stamps, labels3d)
    features_train.append(features)
    labels_train.append(labels)

features_train = np.vstack(features_train)
labels_train = np.hstack(labels_train)

outputPath = os.path.join('..', 'ML', 'train.hdf5')
with h5py.File(outputPath, 'w') as f:
    dset = f.create_dataset('features', data=features_train, compression='gzip')
    dset = f.create_dataset('labels', data=labels_train, compression='gzip')

Number of subjects: 9
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/01
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/02
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/04
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/05
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/06
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/07
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/08
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/10
/Users/umityoruk/Documents/PythonDev/FinalSegmentation/ML/Train/11


In [4]:
inputPath = os.path.join('..', 'ML', 'train.hdf5')
with h5py.File(inputPath) as f:
    features_all = np.array(f['features'])
    labels_all = np.array(f['labels'])

In [5]:
features_all.shape

(1019070, 7)

## Train and save the models

In [6]:
# Select a small number of samples to speed up training
num_select = 10000
np.random.seed(42)
idx_sel = np.random.choice(np.arange(features_all.shape[0]), num_select)
features_sel = features_all[idx_sel,:]
labels_sel = labels_all[idx_sel]

In [7]:
# If features change, run GridSearch again (see cells below) to identify the best hyperparameters.

# Look at the performance of the classifier on the full feature set.
classifier = RandomForestClassifier(n_estimators=100)

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    features_sel, labels_sel, test_size=0.5, random_state=42)

X_mean = np.mean(X_train, axis=0, keepdims=True)
X_std = np.std(X_train, axis=0, keepdims=True)
X_train -= X_mean
X_train /= X_std
X_test -= X_mean
X_test /= X_std

classifier.fit(X_train, y_train)
pred_train = classifier.predict(X_train)
pred_test = classifier.predict(X_test)

print(classification_report(y_train, pred_train))
print(classification_report(y_test, pred_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3228
          1       1.00      1.00      1.00      1060
          2       1.00      1.00      1.00       325
          3       1.00      1.00      1.00       387

avg / total       1.00      1.00      1.00      5000

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3258
          1       0.87      0.93      0.90      1054
          2       0.71      0.58      0.64       308
          3       0.89      0.85      0.87       380

avg / total       0.95      0.95      0.95      5000



In [16]:
# Train models for different feature lengths (needed for shorter datasets)
# The first 6 features are signal samples at t = 0, 30, 60, 90, 120, 150 seconds.
# The next feature is depth of voxel from renal surface.

for num_feats in range(1,8):
#     classifier = SVC(C=100, gamma=0.1)
    classifier = RandomForestClassifier(n_estimators=100)

    # Split the dataset in two equal parts
    X_train, X_test, y_train, y_test = train_test_split(
        features_sel[:,-num_feats:], labels_sel, test_size=0.5, random_state=42)

    X_mean = np.mean(X_train, axis=0, keepdims=True)
    X_std = np.std(X_train, axis=0, keepdims=True)
    X_train -= X_mean
    X_train /= X_std
    X_test -= X_mean
    X_test /= X_std

    classifier.fit(X_train, y_train)
    pred_train = classifier.predict(X_train)
    pred_test = classifier.predict(X_test)

    print(classification_report(y_test, pred_test))
    
    # Save the model
    model = {}
    model['classifier'] = classifier
    model['mean'] = X_mean
    model['std'] = X_std

    model_filename = os.path.join(os.getcwd(), '../Models', 'subsegment_model_f' + str(num_feats) + '.pkl')

    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3258
          1       0.65      0.86      0.74      1054
          2       0.32      0.21      0.25       308
          3       0.36      0.14      0.20       380

avg / total       0.83      0.86      0.84      5000

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3258
          1       0.74      0.84      0.79      1054
          2       0.33      0.23      0.27       308
          3       0.84      0.73      0.78       380

avg / total       0.89      0.90      0.89      5000

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3258
          1       0.77      0.90      0.83      1054
          2       0.43      0.22      0.29       308
          3       0.91      0.82      0.86       380

avg / total       0.91      0.92      0.91      5000

             precision    recall  f1-

In [17]:
# Load the last model to check contents
with open(model_filename, 'rb') as f:
    model2 = pickle.load(f)

In [18]:
model2

{'classifier': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
             verbose=0, warm_start=False),
 'mean': array([[ -2.99793607e-03,   6.31695977e-04,   3.14344195e-04,
          -4.90479065e-03,  -7.97483234e-03,  -1.06558284e-02,
           1.71025663e+00]]),
 'std': array([[ 0.9951846 ,  1.00152891,  1.00201283,  0.99494654,  0.99293985,
          0.98759974,  2.92541207]])}

## Grid Search example for hyperparameter optimization (Random Forests)

In [108]:
num_feats = 7

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    features_sel[:,-num_feats:], labels_sel, test_size=0.5, random_state=42)
X_mean = np.mean(X_train, axis=0, keepdims=True)
X_std = np.std(X_train, axis=0, keepdims=True)
X_train -= X_mean
X_train /= X_std
X_test -= X_mean
X_test /= X_std
X_all = np.vstack((X_train, X_test))
y_all = np.hstack((y_train, y_test))

tuned_parameters = [{'n_estimators': [10, 100, 1000]}]

scores = ['f1']
# scores = ['f1', 'precision', 'recall']

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(RandomForestClassifier(n_estimators=10), tuned_parameters, cv=5, 
                           scoring='%s_weighted' % score)
        
        clf.fit(X_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        for params, mean_score, scores in clf.grid_scores_:
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean_score, scores.std() * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
        print()
        
        
        print("The model is trained on the full development set.")
        print("The scores are computed on the full data set.")
        print()
        y_true, y_pred = y_all, clf.predict(X_all)
        print(classification_report(y_true, y_pred))
        print()

# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'n_estimators': 1000}

Grid scores on development set:

0.904 (+/-0.014) for {'n_estimators': 10}
0.903 (+/-0.014) for {'n_estimators': 100}
0.905 (+/-0.016) for {'n_estimators': 1000}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3258
          1       0.76      0.91      0.83      1054
          2       0.44      0.22      0.29       308
          3       0.91      0.81      0.85       380

avg / total       0.91      0.92      0.91      5000


The model is trained on the full development set.
The scores are computed on the full data set.

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6486
          1       0.87      0.95      0.91      2114
          2      

## Grid Search example for hyperparameter optimization (SVM)

In [11]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    features_sel, labels_sel, test_size=0.5, random_state=3)
X_mean = np.mean(X_train, axis=0, keepdims=True)
X_std = np.std(X_train, axis=0, keepdims=True)
X_train -= X_mean
X_train /= X_std
X_test -= X_mean
X_test /= X_std
X_all = np.vstack((X_train, X_test))
y_all = np.hstack((y_train, y_test))

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [0.01, 0.1, 1],
                     'C': [1, 10, 100]},
#                     {'kernel': ['linear'], 'C': [1, 10]},
                   ]
scores = ['f1']
# scores = ['f1', 'precision', 'recall']

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        if 'custom' in score:
            clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5, scoring=scorer)
        else:
            clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5, 
                               scoring='%s_weighted' % score)
        
        clf.fit(X_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        for params, mean_score, scores in clf.grid_scores_:
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean_score, scores.std() * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = y_test, clf.predict(X_test)
        print(classification_report(y_true, y_pred))
        print()
        
        
        print("The model is trained on the full development set.")
        print("The scores are computed on the full data set.")
        print()
        y_true, y_pred = y_all, clf.predict(X_all)
        print(classification_report(y_true, y_pred))
        print()

# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'gamma': 0.1, 'kernel': 'rbf', 'C': 100}

Grid scores on development set:

0.906 (+/-0.025) for {'gamma': 0.01, 'kernel': 'rbf', 'C': 1}
0.937 (+/-0.012) for {'gamma': 0.1, 'kernel': 'rbf', 'C': 1}
0.937 (+/-0.015) for {'gamma': 1, 'kernel': 'rbf', 'C': 1}
0.937 (+/-0.011) for {'gamma': 0.01, 'kernel': 'rbf', 'C': 10}
0.938 (+/-0.012) for {'gamma': 0.1, 'kernel': 'rbf', 'C': 10}
0.929 (+/-0.012) for {'gamma': 1, 'kernel': 'rbf', 'C': 10}
0.938 (+/-0.012) for {'gamma': 0.01, 'kernel': 'rbf', 'C': 100}
0.941 (+/-0.010) for {'gamma': 0.1, 'kernel': 'rbf', 'C': 100}
0.920 (+/-0.018) for {'gamma': 1, 'kernel': 'rbf', 'C': 100}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3297
          1       0.87      0.94      0.91  